In this notebook, I want to check that the mean orbit as calculated with galpy, agama, and gadget gives the same result.

In [ ]:
using CairoMakie
using Arya
using PythonCall

using DataFrames

In [ ]:
using CSV

In [ ]:
import TOML

In [ ]:
agama = pyimport("agama")
galpy = pyimport("galpy")
gp = pyimport("galpy.potential")
go = pyimport("galpy.orbit")
u = pyimport("astropy.units")
ac = pyimport("astropy.coordinates")
np = pyimport("numpy")

In [ ]:
conversion = galpy.util.conversion

In [ ]:
conversion.mass_in_1010msol(207.4, 1)

In [ ]:
conversion.velocity_in_kpcGyr(207.4, 1)

In [ ]:
MN75 = gp.MiyamotoNagaiPotential
ExpDisk = gp.DoubleExponentialDiskPotential
NFW = gp.NFWPotential
#plaw = gp.PowerSphericalPotentialawCutoff
Plummer = gp.PlummerPotential


EP2020_thin = MN75(amp=5.9, a=3.944, b=0.311)
EP2020_thick =  MN75(amp=2, a=4.4, b=0.92)

M200 = 126.38
c = 11.2849
Ms = LilGuys.NFW(M200=M200, c=c).M_s

EP2020_halo =  NFW(amp=Ms, a=20.2)
EP2020_bulge = gp.HernquistPotential(amp=2.1 * (2), a=1.3) # instead of 1.3

EP2020 = EP2020_thin + EP2020_thick + EP2020_halo + EP2020_bulge

In [ ]:
# agama potential
pot_file = "/astro/dboyea/dwarfs/agama/potentials/EP2020.ini"
Φ_mw_agama = agama.Potential(pot_file)

In [ ]:
obs_props = TOML.parsefile("/astro/dboyea/dwarfs/observations/sculptor/observed_properties.toml")

obs = ICRS(
	distance = obs_props["distance"],
	pmra = obs_props["pmra"] ,
	pmdec = obs_props["pmdec"],
	radial_velocity = obs_props["radial_velocity"],
	ra = obs_props["ra"],
	dec = obs_props["dec"],

)

In [ ]:
obs_gc = LilGuys.transform(LilGuys.Galactocentric, obs)

In [ ]:
# astropy galactocentric


kms = u.km/u.s
sculptor = ac.SkyCoord(ra=obs.ra*u.degree, dec=obs.dec*u.degree, 
                         distance=obs.distance*u.kpc, radial_velocity=obs.radial_velocity*kms,
                         pm_ra_cosdec=obs.pmra*u.mas/u.year, pm_dec=obs.pmdec*u.mas/u.year)  

dwarf_galaxy = sculptor
ac.galactocentric_frame_defaults.set("v4.0")

gc_frame = ac.Galactocentric()
dwarf_galaxy_gc = dwarf_galaxy.transform_to(gc_frame)

print(dwarf_galaxy_gc.cartesian.xyz)
print(dwarf_galaxy_gc.velocity.d_xyz)


In [ ]:
orbit1 = CSV.read("/astro/dboyea/dwarfs/analysis/sculptor/mc_orbits/orbit1.csv", DataFrame)

In [ ]:
T_end = (orbit1.t[1] - orbit1.t[end]) * T2GYR

In [ ]:
np2jl(x) = pyconvert(Vector{Float64}, x)

In [ ]:
o = go.Orbit(dwarf_galaxy_gc, ro=1., vo=207.4, )

o.vxvv

In [ ]:
# have to transform into galpy's coordinates since galpy uses a different frame
# than I do..... :(

In [ ]:
# galpy orbit
o = go.Orbit(np.array(vxvv) )

ts = np.linspace(0, T_end/T2GYR, 100_000)

o.integrate(ts, EP2020,)

df_galpy = DataFrame(
    t = np2jl(ts),
    x = np2jl(o.x(ts)),
    y = np2jl(o.y(ts)),
    z = np2jl(o.z(ts)),
    vx = np2jl(o.vx(ts)) * V2KMS,
    vy = np2jl(o.vy(ts))* V2KMS,
    vz = np2jl(o.vz(ts))* V2KMS,
    
    )

In [ ]:
# agama orbit
phase = obs_gc
posvel = [phase.x, phase.y, phase.z, 
	((phase.v_x, phase.v_y, phase.v_z) ./ V2KMS )...]

orbit_agama = agama.orbit(potential=Φ_mw_agama, ic=posvel, time=T_end/T2GYR, 
    trajsize=1_000, accuracy=1e-12)
t_agama = pyconvert(Vector{Float64}, orbit_agama[0])
posvel_orbit = pyconvert(Matrix{Float64}, orbit_agama[1])'


df_agama = DataFrame(t = t_agama , 
    x=posvel_orbit[1, :], 
    y=posvel_orbit[2, :], 
    z=posvel_orbit[3, :], 
    vx=posvel_orbit[4, :]*V2KMS, 
    vy=posvel_orbit[5, :]*V2KMS,
    vz=posvel_orbit[6, :]*V2KMS
)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="x", ylabel="y")

lines!(df_agama.z, df_agama.y)
lines!(orbit1.z, orbit1.y, linestyle=:dash)
lines!(df_galpy.z, df_galpy.y, linestyle=:dot)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel = "R / kpc", ylabel = "vcirc / km / s")


r = np.logspace(-2, 2, 100)
M = np2jl([gp.mass(EP2020, x) for x in r])
r = np2jl(r)

v = @. sqrt(M / r) * V2KMS

lines!(log10.(r), v)

M = np2jl(Φ_mw_agama.enclosedMass(r))
v = @. sqrt(M / r) * V2KMS
lines!(log10.(r), v, label="agama", linestyle=:dot)


axislegend()
fig
